In [ ]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K

In [ ]:
import tensorflow as tf
tf.__version__
tf.test.gpu_device_name()

In [ ]:
K.clear_session()

data_entrenamiento = './data/train'
data_validacion = './data/validation'

In [ ]:
epocas = 50 #numero de veces q se va a iterar sobre el set de datos.
width_shape = 224
height_shape = 224 # tamano al cual vamos a procesar las imagenes
batch_size = 64 #cantidad de imagenes que enviamos a procesar en cada uno de los pasos
clases=2 #tipo de imagenes q vamos a enviar
lr=0.0005 #determina el tamano de los ajustes que ara nuestra red neuronal

In [ ]:
train_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(width_shape, height_shape),
    batch_size=batch_size,
    class_mode='categorical')
validation_generator = valid_datagen.flow_from_directory(
    data_validacion,
    target_size=(width_shape, height_shape),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
print('#Entrenamiento de modelo VGG16')

nb_train_samples = 10682
nb_validation_samples = 3562

image_input = Input(shape=(width_shape, height_shape, 3))

model2 = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')

model2.summary()

last_layer = model2.get_layer('block5_pool').output
x= Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(clases, activation='softmax', name='output')(x)
custom_model = Model(image_input, out)
custom_model.summary()

# freeze all the layers except the dense layers
for layer in custom_model.layers[:-3]:
    layer.trainable = False

custom_model.summary()

custom_model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

In [ ]:
model_history = custom_model.fit(
    train_generator,
    epochs=epocas,
    validation_data=validation_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    validation_steps=nb_validation_samples//batch_size)